# Wapitify

This notebook serves to emphasize the utilization of the wapitify function, and we presume that the user has already reviewed the step-by-step wapiti tutorial.

In [1]:
from wapiti import wapiti_tools, wapiti

import os

import numpy as np
import pandas as pd

from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.timeseries import LombScargle

from wpca import WPCA

from matplotlib import pyplot as plt
import seaborn

from tqdm.notebook import tqdm as tqdm

seaborn.set_context("notebook")
plt.rcParams["axes.grid"] = False
cwd = os.getcwd()  # current working directory

In [2]:
from wapiti import wapitify, wapiti_tools

seaborn.set_context("notebook")
plt.rcParams["axes.grid"] = False
cwd = os.getcwd()  # current working directory

# Data preprocessing:

## /!\ 


It is assumed that the user possesses LBL per-line radial velocity (RV) timeseries, $\texttt{times_lbl}$, $\texttt{rvs_all}$, and $\texttt{drvs_all}$ as well as drift RVs. 

## /!\ 

Lines with less than 50% of spectra with a valid velocity are removed, and the data are subsequently binned by night.

In [3]:
def get_data(target, template):
    
    tbl = Table.read(cwd+'/lblrv_lam/drift.rdb', format='rdb')
    vrad_drift, svrad_drift, local_file_name = tbl['vrad'].data, tbl['svrad'].data, tbl['local_file_name'].data

    files = os.listdir(cwd+'/lblrv_lam/'+target+'_'+template)

    times_lbl = []
    rvs_all = []
    drvs_all = []
    d2vs_all = []
    dd2vs_all = []
    
    airmass = []
    odocodes = []

    snrs = []
    berv = []
    times_drift = []
    rv_drift = []
    drv_drift = []

    for file in tqdm(files, leave=False):
        lbl = fits.open(cwd+'/lblrv_lam/'+target+'_'+template+'/'+file)
        lbl_data = lbl[1].data
        lbl_header = lbl[0].header
        lbl.close()


        rv_all = lbl_data['dv']
        drv_all = lbl_data['sdv']
        d2v_all = lbl_data['d2v']
        dd2v_all = lbl_data['sd2v']

        odocode = file.split('o')[0]
        
        if lbl_header['BJD']>2458600:
            
            try:
                rv_drift.append(vrad_drift[local_file_name == odocode+'o_pp_e2dsff_C_FP_FP_lbl.fits'][0])
                drv_drift.append(svrad_drift[local_file_name == odocode+'o_pp_e2dsff_C_FP_FP_lbl.fits'][0])
                berv.append(lbl_header['BERV'])

                odocodes.append(odocode)
                airmass.append(lbl_header['AIRMASS'])
                times_drift.append(lbl_header['BJD'])

                times_lbl.append(lbl_header['BJD'])
                rvs_all.append(rv_all)
                drvs_all.append(drv_all)
                d2vs_all.append(d2v_all)
                dd2vs_all.append(dd2v_all)
                snrs.append(lbl_header['SPEMSNR'])
            except:
                pass
    times_lbl = np.array(times_lbl)
    rvs_all = np.array(rvs_all)
    drvs_all = np.array(drvs_all)
    d2vs_all = np.array(d2vs_all)
    dd2vs_all = np.array(dd2vs_all)
    odocodes = np.array(odocodes)

    times_drift = np.array(times_drift)
    rv_drift = np.array(rv_drift)
    drv_drift = np.array(drv_drift)
    
    airmass = np.array(airmass)
    snrs = np.array(snrs)
    berv = np.array(berv)
    waves = (lbl_data['WAVE_START'] + lbl_data['WAVE_END'])/2.
    wave_start = lbl_data['WAVE_START']
    wave_end = lbl_data['WAVE_END']
    
    return times_lbl, rvs_all, drvs_all, d2vs_all, dd2vs_all, odocodes, times_drift, rv_drift, drv_drift, waves, airmass, snrs, berv, wave_start, wave_end

In [4]:
target = 'gl251'
template = target
times_lbl, rvs_all, drvs_all, d2vs_all, dd2vs_all, odocodes, times_drift, rv_drift, drv_drift, waves, airmass, snrs, berv_all, wave_start, wave_end = get_data(target, template)

  0%|          | 0/769 [00:00<?, ?it/s]

In [5]:
g = np.mean(np.isfinite(rvs_all),axis=0)
valid = g >= 0.5
rvs_all, drvs_all = rvs_all[:, valid], drvs_all[:, valid]

In [6]:
rvs_binned = []
drvs_binned = []

for idx in tqdm(range(rvs_all.shape[1]), leave=False):

    time_binned, rv, drv = wapiti_tools.night_bin(times_lbl, rvs_all[:, idx]-rv_drift, np.sqrt(drvs_all[:, idx]**2+drv_drift**2))
    rvs_binned.append(rv)
    drvs_binned.append(drv)

rvs_binned = np.array(rvs_binned).T
drvs_binned = np.array(drvs_binned).T

  0%|          | 0/17887 [00:00<?, ?it/s]

It is feasible to apply the $\texttt{Wapiti}$ method directly by using the $\texttt{wapitify}$ function.

The function will return a dictionnary that encompasses all the products derived from the method.

In [7]:
results_wapiti = wapitify.wapitify(time_binned, rvs_binned, drvs_binned)
results_wapiti

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/173 [00:00<?, ?it/s]

  0%|          | 0/172 [00:00<?, ?it/s]

  0%|          | 0/171 [00:00<?, ?it/s]

...

  0%|          | 0/166 [00:00<?, ?it/s]

  0%|          | 0/165 [00:00<?, ?it/s]

  0%|          | 0/164 [00:00<?, ?it/s]

  0%|          | 0/173 [00:00<?, ?it/s]

{'time_used': array([2458738.14145446, 2458745.12416176, 2458746.13130938,
        2458751.13105619, 2458752.12272485, 2458753.125943  ,
        2458759.1513477 , 2458763.13977478, 2458766.1175354 ,
        2458769.13445571, 2458771.14359075, 2458772.15598238,
        2458773.1299771 , 2458788.12021982, 2458789.12460369,
        2458791.10408415, 2458792.14841385, 2458793.13755229,
        2458795.11102392, 2458797.06359375, 2458798.15112502,
        2458801.10130549, 2458802.07821616, 2458823.03078351,
        2458825.1023752 , 2458826.03472024, 2458827.06536466,
        2458827.98740382, 2458829.09268654, 2458830.08387635,
        2458874.96007405, 2458875.96839383, 2458876.97583548,
        2458884.81609441, 2458885.93584124, 2458895.79517331,
        2458896.86848942, 2458897.79633738, 2458898.83715015,
        2458919.82519007, 2458920.77210018, 2458922.7585227 ,
        2459112.11640603, 2459114.0510637 , 2459115.11574249,
        2459116.10223236, 2459118.08669671, 2459121.01891

Although not advisable, it is also feasible to bypass the selection process for determining a reliable number of components by imposing a predetermined value chosen by the user.

In [8]:
results_wapiti = wapitify.wapitify(time_binned, rvs_binned, drvs_binned, n_components=2)
results_wapiti

  0%|          | 0/174 [00:00<?, ?it/s]

{'time': array([2458738.14145446, 2458745.12416176, 2458746.13130938,
        2458751.13105619, 2458752.12272485, 2458753.125943  ,
        2458759.1513477 , 2458763.13977478, 2458766.1175354 ,
        2458769.13445571, 2458771.14359075, 2458772.15598238,
        2458773.1299771 , 2458788.12021982, 2458789.12460369,
        2458791.10408415, 2458792.14841385, 2458793.13755229,
        2458795.11102392, 2458797.06359375, 2458798.15112502,
        2458801.10130549, 2458802.07821616, 2458823.03078351,
        2458825.1023752 , 2458826.03472024, 2458827.06536466,
        2458827.98740382, 2458829.09268654, 2458830.08387635,
        2458874.96007405, 2458875.96839383, 2458876.97583548,
        2458884.81609441, 2458885.93584124, 2458895.79517331,
        2458896.86848942, 2458897.79633738, 2458898.83715015,
        2458919.82519007, 2458920.77210018, 2458922.7585227 ,
        2459112.11640603, 2459114.0510637 , 2459115.11574249,
        2459116.10223236, 2459118.08669671, 2459121.01891317,
